## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import requests
import json

In [2]:
API_key = '294f4c0e6d0861470f47e84017c8e2c1'

In [50]:
# Imported csv file with movie imdb index to be used to make calls to API.  We would find another table of titles...
df = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
imdb_ID = df['tconst'].unique()

array(['tt0111414', 'tt0323808', 'tt0417610', ..., 'tt9681728',
       'tt9689618', 'tt9692684'], dtype=object)

In [74]:
#Defines a function to then arrange the data into a dictionary for a given movie.
def API_request(movie):
    resp = requests.get('https://api.themoviedb.org/3/movie/' + movie + '?api_key=' + str(API_key))
    if resp.status_code == requests.codes.ok:
        temp = dict(resp.json())
        movie = {}
        movie['imdb_id'] = temp['imdb_id']
        movie['release_date'] = pd.to_datetime(temp['release_date'])
        movie['genres'] = [genre['name'] for genre in temp['genres']]
        movie['title'] = temp['title']
        movie['budget'] = int(temp['budget'])
        movie['revenue'] = int(temp['revenue'])
        movie['ROI'] =  int(temp['revenue']) - int(temp['budget'])
        movie['runtime_mins'] = temp['runtime']
        return movie

In [199]:
# The amount of movies in imdb_ID is 143,454.  Causing the API request to be VERY slow! 
len(imdb_ID)

143454

In [211]:
#Create a subset of the imdb_ID.  
#I think a future state for us to fix, would be to not use all the titles in that file.  We can focus on popular movies?
length = 1000
imdb_ID_subset = imdb_ID[:length]

In [216]:
#Some of the titles do not have data from the API: See example below:
print(API_request(imdb_ID[0]))

None


In [212]:
#Some of the API requests return a 'None' value, which means there is no information for that title.  
#This fucntion cleans the titles to remove titles that do not have any information from the API
imdb_ID_filtered = list(filter(lambda x: API_request(x) != None ,imdb_ID_subset))

In [213]:
imbd_API_data = list(map(lambda x: API_request(x),imdb_ID_filtered))

In [214]:
#There is still a little more that needs to be done.  There are a bunch of movies that have 0 values which we can drop.
#I'm not sure the movies in this list are the best for the analysis, want to chat with the team around that.  
#Maybe look at another file and target the most popular movies?
#There was discussions around us breaking down the data by year.

df = pd.DataFrame.from_dict(imbd_API_data)
df

,imdb_id,release_date,genres,title,budget,revenue,ROI,runtime_mins
0,tt0323808,2011-08-27,"[Mystery, Horror, Thriller]",The Wicker Tree,7750000,0,-7750000,96.0
1,tt0417610,2011-04-24,[Fantasy],Asleep in the Sun,0,0,0,83.0
2,tt0469152,2011-10-13,[Drama],The Pack,0,0,0,84.0
3,tt0475290,2016-02-05,"[Comedy, Drama, Mystery]","Hail, Caesar!",22000000,63647656,41647656,106.0
4,tt0477302,2011-12-24,[Drama],Extremely Loud & Incredibly Close,40000000,55247881,15247881,129.0
...,...,...,...,...,...,...,...,...
615,tt1863262,2010-01-01,"[Documentary, Drama, History]",Hatalyan (The Hangman),0,0,0,60.0
616,tt1863426,2010-08-31,[],The Occult: The Truth Behind the Word,0,0,0,0.0
617,tt1864368,2011-03-18,[Documentary],Indian Summer,0,0,0,70.0
618,tt1865346,2012-04-27,"[Comedy, Horror]",Detention of the Dead,500000,0,-500000,87.0


In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   imdb_id       620 non-null    object        
 1   release_date  592 non-null    datetime64[ns]
 2   genres        620 non-null    object        
 3   title         620 non-null    object        
 4   budget        620 non-null    int64         
 5   revenue       620 non-null    int64         
 6   ROI           620 non-null    int64         
 7   runtime_mins  580 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 38.9+ KB


In [4]:
resp = requests.get('https://api.themoviedb.org/3/search/movie?api_key=' + str(API_key) + '&query=2017')

In [5]:
dict(resp.json())

{'page': 1,
 'total_results': 575,
 'total_pages': 29,
 'results': [{'popularity': 7.175,
   'id': 449674,
   'video': False,
   'vote_count': 155,
   'vote_average': 7.4,
   'title': 'Louis C.K.: 2017',
   'release_date': '2017-04-04',
   'original_language': 'en',
   'original_title': 'Louis C.K.: 2017',
   'genre_ids': [35],
   'backdrop_path': '/1rRoyJJBr2QceI1OMCSODy77mwq.jpg',
   'adult': False,
   'overview': 'Louis C.K. muses on religion, eternal love, giving dogs drugs, email fights, teachers and more in a live performance from Washington, D.C.',
   'poster_path': '/RlENlbNxrqsWfdEDCSXGWWvWTd.jpg'},
  {'popularity': 6.957,
   'vote_count': 12,
   'video': True,
   'poster_path': '/vscoGNZ8Xt5UZKzFFWDLXbU48Md.jpg',
   'id': 467698,
   'adult': False,
   'backdrop_path': '/sVNjWkMLEEWAG38bwCy5dt4LBSG.jpg',
   'original_language': 'en',
   'original_title': 'WWE Battleground 2017',
   'genre_ids': [],
   'title': 'WWE Battleground 2017',
   'vote_average': 6.7,
   'overview': 'Ba

In [6]:
df = pd.DataFrame.from_dict(dict(resp.json()))
df


,page,total_results,total_pages,results
0,1,575,29,"{'popularity': 7.175, 'id': 449674, 'video': F..."
1,1,575,29,"{'popularity': 6.957, 'vote_count': 12, 'video..."
2,1,575,29,"{'popularity': 7.532, 'vote_count': 13, 'video..."
3,1,575,29,"{'popularity': 5.116, 'id': 455362, 'video': T..."
4,1,575,29,"{'popularity': 5.593, 'vote_count': 14, 'video..."
5,1,575,29,"{'popularity': 8.438, 'vote_count': 11, 'video..."
6,1,575,29,"{'popularity': 4.201, 'vote_count': 16, 'video..."
7,1,575,29,"{'popularity': 6.776, 'vote_count': 11, 'video..."
8,1,575,29,"{'popularity': 4.855, 'id': 441436, 'video': T..."
9,1,575,29,"{'popularity': 4.575, 'vote_count': 18, 'video..."


In [19]:
test = dict(resp.json())
test

{'page': 1,
 'total_results': 575,
 'total_pages': 29,
 'results': [{'popularity': 7.175,
   'id': 449674,
   'video': False,
   'vote_count': 155,
   'vote_average': 7.4,
   'title': 'Louis C.K.: 2017',
   'release_date': '2017-04-04',
   'original_language': 'en',
   'original_title': 'Louis C.K.: 2017',
   'genre_ids': [35],
   'backdrop_path': '/1rRoyJJBr2QceI1OMCSODy77mwq.jpg',
   'adult': False,
   'overview': 'Louis C.K. muses on religion, eternal love, giving dogs drugs, email fights, teachers and more in a live performance from Washington, D.C.',
   'poster_path': '/RlENlbNxrqsWfdEDCSXGWWvWTd.jpg'},
  {'popularity': 6.957,
   'vote_count': 12,
   'video': True,
   'poster_path': '/vscoGNZ8Xt5UZKzFFWDLXbU48Md.jpg',
   'id': 467698,
   'adult': False,
   'backdrop_path': '/sVNjWkMLEEWAG38bwCy5dt4LBSG.jpg',
   'original_language': 'en',
   'original_title': 'WWE Battleground 2017',
   'genre_ids': [],
   'title': 'WWE Battleground 2017',
   'vote_average': 6.7,
   'overview': 'Ba

In [20]:
test['total_pages']

29

In [6]:
resp1 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' + str(API_key) + '&query=2020&page=8')
test2 = dict(resp1.json())
test2

{'page': 8,
 'total_results': 10000,
 'total_pages': 500,
 'results': [{'popularity': 36.266,
   'vote_count': 4648,
   'video': False,
   'poster_path': '/rtGDOeG9LzoerkDGZF9dnVeLppL.jpg',
   'id': 8392,
   'adult': False,
   'backdrop_path': '/95ozIP0A2fKaAXxwDxUEVn74Iux.jpg',
   'original_language': 'ja',
   'original_title': 'となりのトトロ',
   'genre_ids': [16, 14, 10751],
   'title': 'My Neighbor Totoro',
   'vote_average': 8.1,
   'overview': 'Two sisters move to the country with their father in order to be closer to their hospitalized mother, and discover the surrounding trees are inhabited by Totoros, magical spirits of the forest. When the youngest runs away from home, the older sister seeks help from the spirits to find her.',
   'release_date': '1988-04-16'},
  {'popularity': 35.934,
   'vote_count': 7790,
   'video': False,
   'poster_path': '/c9XxwwhPHdaImA2f1WEfEsbhaFB.jpg',
   'id': 351286,
   'adult': False,
   'backdrop_path': '/3s9O5af2xWKWR5JzP2iJZpZeQQg.jpg',
   'origina

In [42]:
test2['results']

[{'popularity': 152.206,
  'vote_count': 102,
  'video': False,
  'poster_path': '/hPkqY2EMqWUnFEoedukilIUieVG.jpg',
  'id': 531876,
  'adult': False,
  'backdrop_path': '/n1RohH2VoK1CdVI2fXvcP19dSlm.jpg',
  'original_language': 'en',
  'original_title': 'The Outpost',
  'genre_ids': [28, 18, 36, 10752],
  'title': 'The Outpost',
  'vote_average': 7,
  'overview': 'A small unit of U.S. soldiers, alone at the remote Combat Outpost Keating, located deep in the valley of three mountains in Afghanistan, battles to defend against an overwhelming force of Taliban fighters in a coordinated attack. The Battle of Kamdesh, as it was known, was the bloodiest American engagement of the Afghan War in 2009 and Bravo Troop 3-61 CAV became one of the most decorated units of the 19-year conflict.',
  'release_date': '2020-06-24'},
 {'popularity': 180.174,
  'vote_count': 3905,
  'video': False,
  'poster_path': '/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg',
  'id': 419704,
  'adult': False,
  'backdrop_path': '/A

In [76]:
resp1 = requests.get('https://api.themoviedb.org/3/genre/movie/list?api_key=' + str(API_key)+'&language=en-US')
test3 = dict(resp1.json())
test3['genres']

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [82]:
genres = {}
for x in test3['genres']:
        genres[x['id']] = x['name']

In [83]:
genres

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [80]:
genres[28]

'Action'

In [68]:
search_year = 2017
resp = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=' + str(API_key) +'&language=en-US&sort_by=popularity.desc&primary_release_year='+ str(search_year))
data = dict(resp.json())
search_pages = data['total_pages']
df = pd.DataFrame.from_dict(data['results'])
df

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,41.533,6448,False,/b9CeobiihCx1uG1tpw8hXmpi7nm.jpg,316029,False,/zpq404Sk7qQ7N4x3xOeNgp74GtU.jpg,en,The Greatest Showman,[18],The Greatest Showman,8.0,"The story of American showman P.T. Barnum, fou...",2017-12-20
1,55.385,14499,False,/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg,315635,False,/vc8bCGjdVp0UbMNLzHnHSLRbBWQ.jpg,en,Spider-Man: Homecoming,"[28, 12, 878, 18]",Spider-Man: Homecoming,7.4,Following the events of Captain America: Civil...,2017-07-05
2,51.271,9275,False,/eifGNCSDuxJeS1loAXil5bIGgvC.jpg,141052,False,/jorgjEk6a0bed6jdR5wu4S6ZvRm.jpg,en,Justice League,"[28, 12, 14, 878]",Justice League,6.2,Fuelled by his restored faith in humanity and ...,2017-11-15
3,40.410,14885,False,/gfJGlDaHuWimErCr5Ql0I8x9QSy.jpg,297762,False,/6iUNJZymJBMXXriQyFZfLAKnjO6.jpg,en,Wonder Woman,"[28, 12, 14]",Wonder Woman,7.3,An Amazon princess comes to the world of Man i...,2017-05-30
4,41.535,14199,False,/rzRwTcFvttcN1ZpX2xv4j3tSdJu.jpg,284053,False,/kaIfm5ryEOwYg8mLbq8HkPuM1Fo.jpg,en,Thor: Ragnarok,"[28, 12, 35, 14]",Thor: Ragnarok,7.6,Thor is imprisoned on the other side of the un...,2017-10-25
5,45.683,8463,False,/gajva2L0rPYkEWjzgFlBXCAVBE5.jpg,335984,False,/sAtoMqDVhNDQBc3QJL3RF6hlhGq.jpg,en,Blade Runner 2049,"[18, 878]",Blade Runner 2049,7.4,Thirty years after the events of the first fil...,2017-10-04
6,39.142,15091,False,/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,283995,False,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg,en,Guardians of the Galaxy Vol. 2,"[28, 12, 35, 878]",Guardians of the Galaxy Vol. 2,7.6,The Guardians must fight to keep their newfoun...,2017-04-19
7,31.809,11681,False,/eKi8dIrr8voobbaGzDpe8w0PVbC.jpg,354912,False,/gKZAQ3b9yApxFunUBlS5Mp74QG4.jpg,en,Coco,"[12, 16, 35, 14, 10751]",Coco,8.2,Despite his family’s baffling generations-old ...,2017-10-27
8,35.376,10741,False,/kOVEVeg59E0wsnXmF9nrh6OmWII.jpg,181808,False,/5Iw7zQTHVRBOYpA0V6z0yypOPZh.jpg,en,Star Wars: The Last Jedi,"[28, 12, 878]",Star Wars: The Last Jedi,7.0,Rey develops her newly discovered abilities wi...,2017-12-13
9,28.506,8026,False,/hXWBc0ioZP3cN4zCu6SN3YHXZVO.jpg,324552,False,/fRLftkwv8jm3DLxXOf2NbngHjPB.jpg,en,John Wick: Chapter 2,"[28, 80, 53]",John Wick: Chapter 2,7.2,John Wick is forced out of retirement by a for...,2017-02-08


In [69]:
#This step is slow, there is a lot of data being requested.
url ='https://api.themoviedb.org/3/discover/movie?api_key=' + str(API_key) +'&language=en-US&sort_by=popularity.desc&primary_release_year='+ str(search_year)+'&page='
for page in range(2,search_pages+1):
    df_by_page
    resp = requests.get(url + str(page))
    data = dict(resp.json())
    df_by_page = pd.DataFrame.from_dict(data['results'])
    df = pd.concat([df,df_by_page], ignore_index=True)

df

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,41.533,6448,False,/b9CeobiihCx1uG1tpw8hXmpi7nm.jpg,316029,False,/zpq404Sk7qQ7N4x3xOeNgp74GtU.jpg,en,The Greatest Showman,[18],The Greatest Showman,8.0,"The story of American showman P.T. Barnum, fou...",2017-12-20
1,55.385,14499,False,/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg,315635,False,/vc8bCGjdVp0UbMNLzHnHSLRbBWQ.jpg,en,Spider-Man: Homecoming,"[28, 12, 878, 18]",Spider-Man: Homecoming,7.4,Following the events of Captain America: Civil...,2017-07-05
2,51.271,9275,False,/eifGNCSDuxJeS1loAXil5bIGgvC.jpg,141052,False,/jorgjEk6a0bed6jdR5wu4S6ZvRm.jpg,en,Justice League,"[28, 12, 14, 878]",Justice League,6.2,Fuelled by his restored faith in humanity and ...,2017-11-15
3,40.410,14885,False,/gfJGlDaHuWimErCr5Ql0I8x9QSy.jpg,297762,False,/6iUNJZymJBMXXriQyFZfLAKnjO6.jpg,en,Wonder Woman,"[28, 12, 14]",Wonder Woman,7.3,An Amazon princess comes to the world of Man i...,2017-05-30
4,41.535,14199,False,/rzRwTcFvttcN1ZpX2xv4j3tSdJu.jpg,284053,False,/kaIfm5ryEOwYg8mLbq8HkPuM1Fo.jpg,en,Thor: Ragnarok,"[28, 12, 35, 14]",Thor: Ragnarok,7.6,Thor is imprisoned on the other side of the un...,2017-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.600,0,False,/wO8EV7KLlLUBT2kA9t9WkmYi5wm.jpg,481800,False,None,de,Nicht mit uns! Der Silikon-Skandal,[18],Nicht mit uns! Der Silikon-Skandal,0.0,Three women go to court after their breast imp...,2017-10-17
9996,0.600,0,False,/83Caca2LYJ1O0i7VT04kBbZkXPC.jpg,481690,False,None,fr,Around Luisa,[],Around Luisa,0.0,,2017-10-19
9997,0.600,0,False,/fVcjb7sLs0nJqoQ2Z44b3k8GcdR.jpg,481655,False,None,pt,Utopia,[99],Utopia,0.0,,2017-10-22
9998,0.608,0,False,/jfl4tzUKbYR5qe0uem7qIFZNqpz.jpg,481578,False,/9T5yFn6NglXmnEdIF2FYWac82dL.jpg,hi,Call For Fun,[35],Call For Fun,0.0,Yash incurs heavy losses in the business that ...,2017-10-07


In [70]:
df1 = df.sort_values(by='popularity', ascending=False)
df1 = df1.set_index('id')
df3 = df1.drop(columns = 'genre_ids')
df5 = df3.drop_duplicates()
df5
len(df1)
# df2 = df1.drop_duplicates()

10000

In [85]:
df3 = df1.drop(columns = 'genre_ids')
df5 = df3.drop_duplicates()
df5

,popularity,vote_count,video,poster_path,adult,backdrop_path,original_language,original_title,title,vote_average,overview,release_date
id,,,,,,,,,,,,
315635,55.385,14499,False,/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg,False,/vc8bCGjdVp0UbMNLzHnHSLRbBWQ.jpg,en,Spider-Man: Homecoming,Spider-Man: Homecoming,7.4,Following the events of Captain America: Civil...,2017-07-05
141052,51.271,9275,False,/eifGNCSDuxJeS1loAXil5bIGgvC.jpg,False,/jorgjEk6a0bed6jdR5wu4S6ZvRm.jpg,en,Justice League,Justice League,6.2,Fuelled by his restored faith in humanity and ...,2017-11-15
335984,45.683,8463,False,/gajva2L0rPYkEWjzgFlBXCAVBE5.jpg,False,/sAtoMqDVhNDQBc3QJL3RF6hlhGq.jpg,en,Blade Runner 2049,Blade Runner 2049,7.4,Thirty years after the events of the first fil...,2017-10-04
284053,41.535,14199,False,/rzRwTcFvttcN1ZpX2xv4j3tSdJu.jpg,False,/kaIfm5ryEOwYg8mLbq8HkPuM1Fo.jpg,en,Thor: Ragnarok,Thor: Ragnarok,7.6,Thor is imprisoned on the other side of the un...,2017-10-25
316029,41.533,6448,False,/b9CeobiihCx1uG1tpw8hXmpi7nm.jpg,False,/zpq404Sk7qQ7N4x3xOeNgp74GtU.jpg,en,The Greatest Showman,The Greatest Showman,8.0,"The story of American showman P.T. Barnum, fou...",2017-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...
553122,0.600,0,False,/onNlul9q1a6RN9L3jZ2DMr7Coqg.jpg,False,None,ms,Hajat,Hajat,0.0,The story of a grandson who tried to fulfill h...,2017-10-14
569985,0.600,0,False,/nXbfTIbmzICsYg3IgUBY9wzdkAS.jpg,False,None,en,Go to Hell!,Go to Hell!,0.0,GO TO HELL! follows the triplet Leibowitz brot...,2017-10-27
575327,0.600,0,False,/mo1v3F2HCKP2Rm11OaQJLoPtx1I.jpg,False,None,fa,Abji,Sister,0.0,Tala and Ati (Sis) are a mother and daughter w...,2017-06-02


In [76]:
print(API_request(str(269149)))

{'imdb_id': 'tt2948356', 'release_date': Timestamp('2016-02-11 00:00:00'), 'genres': ['Animation', 'Adventure', 'Family', 'Comedy'], 'title': 'Zootopia', 'budget': 150000000, 'revenue': 1023784195, 'ROI': 873784195, 'runtime_mins': 109}


In [91]:
movies = df5.index.values.tolist()